In [7]:
import json
from collections import Counter

with open("../../data/cv2_transfer/instances_slicks_test_v2.json") as annotation:
    data=json.load(annotation)

for x in data:
    print(x)

info
licenses
images
annotations
categories


In [58]:
list(data['categories'])

[{'supercategory': 'slick', 'id': 1, 'name': 'infra_slick'},
 {'supercategory': 'slick', 'id': 2, 'name': 'natural_seep'},
 {'supercategory': 'slick', 'id': 3, 'name': 'coincident_vessel'},
 {'supercategory': 'slick', 'id': 4, 'name': 'recent_vessel'},
 {'supercategory': 'slick', 'id': 5, 'name': 'old_vessel'},
 {'supercategory': 'slick', 'id': 6, 'name': 'ambiguous'}]

In [30]:
infra_slick_stats = [] 
natural_seep_stats = []
coincident_vessel_stats = []
recent_vessel_stats = []
old_vessel_stats = []
ambiguous_stats = []

In [31]:
for c in list(data['annotations']):
    cat = c['category_id']
    if cat == 1:
        infra_slick_stats.append(c)
    elif cat == 2:
        natural_seep_stats.append(c)
    elif cat == 3:
        coincident_vessel_stats.append(c)
    elif cat == 4:
        recent_vessel_stats.append(c)
    elif cat == 5:
        old_vessel_stats.append(c)
    elif cat == 6:
        ambiguous_stats.append(c)                          

In [34]:
len(infra_slick_stats), len(natural_seep_stats), len(coincident_vessel_stats), len(recent_vessel_stats), len(old_vessel_stats), len(ambiguous_stats)

(31, 263, 100, 3, 80, 10)

In [39]:
infra_slick_stats_area = [] 
natural_seep_stats_area = []
coincident_vessel_stats_area = []
recent_vessel_stats_area = []
old_vessel_stats_area = []
ambiguous_stats_area = []

In [40]:
for a in infra_slick_stats:
    area = a['area']
    infra_slick_stats_area.append(area)
for a in natural_seep_stats:
    area = a['area']
    natural_seep_stats_area.append(area)
for a in coincident_vessel_stats:
    area = a['area']
    coincident_vessel_stats_area.append(area)
for a in recent_vessel_stats:
    area = a['area']
    recent_vessel_stats_area.append(area)
for a in old_vessel_stats:
    area = a['area']
    old_vessel_stats_area.append(area)
for a in ambiguous_stats:
    area = a['area']
    ambiguous_stats_area.append(area)

In [41]:
sum(infra_slick_stats_area), sum(natural_seep_stats_area), sum(coincident_vessel_stats_area), sum(recent_vessel_stats_area), sum(old_vessel_stats_area), sum(ambiguous_stats_area) 

(91335, 1360326, 182448, 2760, 160981, 34593)

In [55]:
infra_slick_stats_maxsidelen = [] 
natural_seep_stats_maxsidelen = []
coincident_vessel_stats_maxsidelen = []
recent_vessel_stats_maxsidelen = []
old_vessel_stats_maxsidelen = []
ambiguous_stats_maxsidelen = []

In [56]:
for l in infra_slick_stats:
    ml = l['bbox']
    ml1 = ml[2]-ml[0]
    ml2 = ml[3]-ml[1]
    ml3 = max(ml1, ml2)
    infra_slick_stats_maxsidelen.append(abs(ml3))
for l in natural_seep_stats:
    ml = l['bbox']
    ml1 = ml[2]-ml[0]
    ml2 = ml[3]-ml[1]
    ml3 = max(ml1, ml2)
    natural_seep_stats_maxsidelen.append(abs(ml3))
for l in coincident_vessel_stats:
    ml = l['bbox']
    ml1 = ml[2]-ml[0]
    ml2 = ml[3]-ml[1]
    ml3 = max(ml1, ml2)
    coincident_vessel_stats_maxsidelen.append(abs(ml3))
for l in recent_vessel_stats:
    ml = l['bbox']
    ml1 = ml[2]-ml[0]
    ml2 = ml[3]-ml[1]
    ml3 = max(ml1, ml2)
    recent_vessel_stats_maxsidelen.append(abs(ml3))
for l in old_vessel_stats:
    ml = l['bbox']
    ml1 = ml[2]-ml[0]
    ml2 = ml[3]-ml[1]
    ml3 = max(ml1, ml2)
    old_vessel_stats_maxsidelen.append(abs(ml3))
for l in ambiguous_stats:
    ml = l['bbox']
    ml1 = ml[2]-ml[0]
    ml2 = ml[3]-ml[1]
    ml3 = max(ml1, ml2)
    ambiguous_stats_maxsidelen.append(abs(ml3))

In [57]:
max(infra_slick_stats_maxsidelen), max(natural_seep_stats_maxsidelen),max(coincident_vessel_stats_maxsidelen),max(recent_vessel_stats_maxsidelen),max(old_vessel_stats_maxsidelen),max(ambiguous_stats_maxsidelen)


(466.0, 512.0, 512.0, 181.0, 512.0, 221.0)